## Importing Dependencies

In [11]:
import pandas as pd

from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy.types import (Date, DateTime, Float, Integer,
                              String, Interval, Time)

## Connecting to SQLite DB

In [12]:
# Creating the SQLITE DB
db_path = 'Resources/Flights_DB.sqlite'
engine = create_engine('sqlite:///{database_path}')

# Connecting to engine
engine.connect()

## Creating the Flights table

In [4]:
# Import Flights table as DataFrame
flights_path = Path('Resources/DB_Flights_Table.csv')
flights_df = pd.read_csv(flights_path, index_col=0)

flights_df.head(10)

,Airline_Flight_Number,Operating_Airline,Origin_Airport,Destination_Airport,Flight_Date,Departure_Time,Arrival_Time,Delayed,Cancelled
0,1581,DL,FLL,LGA,2022-01-06,NaN,NaN,0,1.0
1,1582,DL,ATL,FLL,2022-01-06,16:27,18:20,0,0.0
2,1582,DL,FLL,ATL,2022-01-06,19:29,21:15,0,0.0
3,1583,DL,FLL,RDU,2022-01-06,10:19,12:12,0,0.0
4,1584,DL,ATL,JAN,2022-01-06,11:13,11:31,0,0.0
5,1584,DL,JAN,ATL,2022-01-06,12:30,14:35,0,0.0
6,1585,DL,RIC,ATL,2022-01-06,08:57,10:38,0,0.0
7,1586,DL,MSP,RSW,2022-01-06,10:18,14:35,1,0.0
8,1587,DL,ATL,RDU,2022-01-06,14:22,15:31,1,0.0
9,1587,DL,RDU,ATL,2022-01-06,16:40,18:03,0,0.0


In [16]:
# Convert DataFrame to SQLITE table
flights_df.to_sql('Flights',
                  con=engine,
                  index=True,
                  index_label='ID',
                  if_exists='replace',
                  dtype={
                      'Airline_Flight_Number': Integer(),
                      'Operating_Airline': String(),
                      'Origin_Airport': String(),
                      'Destination_Airport': String(),
                      'Flight_Date': String(),
                      'Departure_Time': String(),
                      'Arrival_Time': String(),
                      'Delayed': Integer(),
                      'Cancelled': Integer()
                   })

4078318

In [17]:
# Check if successful
engine.execute('SELECT * FROM Flights LIMIT 5').fetchall()

[(0, 1581, 'DL', 'FLL', 'LGA', '2022-01-06', None, None, 0, 1),
 (1, 1582, 'DL', 'ATL', 'FLL', '2022-01-06', '16:27', '18:20', 0, 0),
 (2, 1582, 'DL', 'FLL', 'ATL', '2022-01-06', '19:29', '21:15', 0, 0),
 (3, 1583, 'DL', 'FLL', 'RDU', '2022-01-06', '10:19', '12:12', 0, 0),
 (4, 1584, 'DL', 'ATL', 'JAN', '2022-01-06', '11:13', '11:31', 0, 0)]

In [18]:
# Read DataFrame back from SQLite
pd.read_sql('SELECT * FROM Flights LIMIT 5',
            con=engine,
            index_col='ID',
            parse_dates=['Flight_Date', 'Departure_Time', 'Arrival_Time'])

,Airline_Flight_Number,Operating_Airline,Origin_Airport,Destination_Airport,Flight_Date,Departure_Time,Arrival_Time,Delayed,Cancelled
ID,,,,,,,,,
0,1581,DL,FLL,LGA,2022-01-06,NaT,NaT,0,1
1,1582,DL,ATL,FLL,2022-01-06,2023-02-08 16:27:00,2023-02-08 18:20:00,0,0
2,1582,DL,FLL,ATL,2022-01-06,2023-02-08 19:29:00,2023-02-08 21:15:00,0,0
3,1583,DL,FLL,RDU,2022-01-06,2023-02-08 10:19:00,2023-02-08 12:12:00,0,0
4,1584,DL,ATL,JAN,2022-01-06,2023-02-08 11:13:00,2023-02-08 11:31:00,0,0
